In [2]:
import torch

class Dummy:
    pass
args = Dummy
args.spec = "test_cases/fc1/img0_0.06000.txt"

with open(args.spec, 'r') as f:
    lines = [line[:-1] for line in f.readlines()]
    true_label = int(lines[0])
    pixel_values = [float(line) for line in lines[1:]]
    eps = float(args.spec[:-4].split('/')[-1].split('_')[-1])
    
DEVICE = 'cpu'
INPUT_SIZE = 28
inp = torch.FloatTensor(pixel_values).view(1, 1, INPUT_SIZE, INPUT_SIZE).to(DEVICE)

inp.shape

torch.Size([1, 1, 28, 28])

## Checking the initialization of a0 and A

In [3]:
eps = 0.35

inp = torch.rand(1, 1, 5, 5)
inp

tensor([[[[8.2404e-02, 7.5609e-02, 8.5135e-01, 2.7218e-01, 9.3328e-01],
          [2.2472e-01, 4.5968e-01, 4.0404e-01, 5.0873e-01, 4.1913e-03],
          [8.3949e-01, 2.4498e-05, 7.1681e-01, 4.3749e-01, 3.1196e-01],
          [1.3487e-01, 6.7814e-02, 3.0574e-01, 5.9705e-02, 2.3245e-01],
          [9.3316e-01, 6.9563e-01, 6.0431e-01, 3.8349e-01, 9.3682e-01]]]])

In [4]:
upper = inp + eps
lower = inp - eps
upper.clamp_(max=1) # clip input_zonotope to the input space
lower.clamp_(min=0)
a0 = (upper + lower) / 2 # center of the zonotope

print(upper)
print(lower)
print(a0.shape)

tensor([[[[0.4324, 0.4256, 1.0000, 0.6222, 1.0000],
          [0.5747, 0.8097, 0.7540, 0.8587, 0.3542],
          [1.0000, 0.3500, 1.0000, 0.7875, 0.6620],
          [0.4849, 0.4178, 0.6557, 0.4097, 0.5825],
          [1.0000, 1.0000, 0.9543, 0.7335, 1.0000]]]])
tensor([[[[0.0000, 0.0000, 0.5013, 0.0000, 0.5833],
          [0.0000, 0.1097, 0.0540, 0.1587, 0.0000],
          [0.4895, 0.0000, 0.3668, 0.0875, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.5832, 0.3456, 0.2543, 0.0335, 0.5868]]]])
torch.Size([1, 1, 5, 5])
torch.Size([5, 5])


In [28]:
# A must have shape (nb_error_terms, *[shape of input])
# for the input layer, there is 1 error term for each pixel, so nb_error_terms = inp.numel()
A = torch.zeros(inp.numel(), *inp.shape[1:])

mask = torch.ones(1, 5, 5, dtype=torch.bool)
A[:, mask] = torch.diag( ((upper - lower) / 2).reshape(-1) )
A

tensor([[[[0.2162, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.2128, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.2493, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000, 0.3111, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],


## Checking the relu() update (new error terms)

In [67]:
my_dim = torch.Size([2, 5, 5]) # 2 channels, width=height=5
my_nb_error_terms = 1 # 1 error terms initially
A = 100*torch.rand(my_nb_error_terms, *my_dim)

B = torch.rand_like(A)
apn = (B < 0.05)[0] # approx_neurons boolean map, i.e these are the neurons that need

# we add 1 error term per neuron to approximate
nb_new_error_terms = apn.nonzero().size(0)
print("nb_new_error_terms:", nb_new_error_terms)
A = torch.cat([A, torch.zeros(nb_new_error_terms, *my_dim)], dim=0)

A_bak = A.clone().detach() # make a backup...

print(A.shape)
A[my_nb_error_terms:]

nb_new_error_terms: 2
torch.Size([3, 2, 5, 5])


tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]],

         [[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]],

         [[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]]])

In [68]:
lambdas = torch.zeros(1, *my_dim)
print(lambdas.shape)
lambdas[:, apn] = 5000
lambdas

torch.Size([1, 2, 5, 5])


tensor([[[[   0.,    0.,    0.,    0.,    0.],
          [   0.,    0.,    0.,    0.,    0.],
          [   0.,    0.,    0.,    0.,    0.],
          [   0.,    0.,    0.,    0.,    0.],
          [   0.,    0.,    0.,    0.,    0.]],

         [[   0.,    0.,    0.,    0.,    0.],
          [5000.,    0.,    0.,    0.,    0.],
          [   0.,    0.,    0.,    0.,    0.],
          [   0., 5000.,    0.,    0.,    0.],
          [   0.,    0.,    0.,    0.,    0.]]]])

In [69]:
# update the old epsilons
torch.set_printoptions(sci_mode=False, threshold=10000, precision=1)
A[:my_nb_error_terms, apn] = A_bak[:my_nb_error_terms, apn] * (lambdas[:, apn])
print("old A\n", A_bak)
print("updated A\n", A)

old A
 tensor([[[[   29.4,    60.8,    84.4,    40.1,    81.5],
          [   31.3,    65.5,     0.1,    27.6,    69.6],
          [   18.2,    74.3,    58.7,    12.3,    73.6],
          [   52.3,    81.9,    14.0,    53.2,    58.2],
          [   34.1,    93.2,    17.0,    90.0,    52.1]],

         [[   17.9,    23.0,    22.0,    76.9,    39.3],
          [   69.8,    31.4,    35.3,    73.8,     5.0],
          [   79.7,     9.2,    72.7,    79.8,    46.6],
          [   47.3,    91.3,    52.3,    96.3,    45.7],
          [    7.5,    61.3,     6.7,    70.5,    93.4]]],


        [[[    0.0,     0.0,     0.0,     0.0,     0.0],
          [    0.0,     0.0,     0.0,     0.0,     0.0],
          [    0.0,     0.0,     0.0,     0.0,     0.0],
          [    0.0,     0.0,     0.0,     0.0,     0.0],
          [    0.0,     0.0,     0.0,     0.0,     0.0]],

         [[    0.0,     0.0,     0.0,     0.0,     0.0],
          [    0.0,     0.0,     0.0,     0.0,     0.0],
          [    0

In [90]:
mu = torch.zeros(1, *my_dim)
print(mu.shape)
mu[:, apn] = torch.tensor([[1000., 2000.]]) # assumes that nb_new_error_terms = 2.
mu

torch.Size([1, 2, 5, 5])


tensor([[[[   0.,    0.,    0.,    0.,    0.],
          [   0.,    0.,    0.,    0.,    0.],
          [   0.,    0.,    0.,    0.,    0.],
          [   0.,    0.,    0.,    0.,    0.],
          [   0.,    0.,    0.,    0.,    0.]],

         [[   0.,    0.,    0.,    0.,    0.],
          [1000.,    0.,    0.,    0.,    0.],
          [   0.,    0.,    0.,    0.,    0.],
          [   0., 2000.,    0.,    0.,    0.],
          [   0.,    0.,    0.,    0.,    0.]]]])

In [91]:
# populate the new epsilon coefficients
A[my_nb_error_terms:, apn] = torch.diag(mu[:, apn].reshape(-1))
A

tensor([[[[   29.4,    60.8,    84.4,    40.1,    81.5],
          [   31.3,    65.5,     0.1,    27.6,    69.6],
          [   18.2,    74.3,    58.7,    12.3,    73.6],
          [   52.3,    81.9,    14.0,    53.2,    58.2],
          [   34.1,    93.2,    17.0,    90.0,    52.1]],

         [[   17.9,    23.0,    22.0,    76.9,    39.3],
          [349070.7,    31.4,    35.3,    73.8,     5.0],
          [   79.7,     9.2,    72.7,    79.8,    46.6],
          [   47.3, 456641.9,    52.3,    96.3,    45.7],
          [    7.5,    61.3,     6.7,    70.5,    93.4]]],


        [[[    0.0,     0.0,     0.0,     0.0,     0.0],
          [    0.0,     0.0,     0.0,     0.0,     0.0],
          [    0.0,     0.0,     0.0,     0.0,     0.0],
          [    0.0,     0.0,     0.0,     0.0,     0.0],
          [    0.0,     0.0,     0.0,     0.0,     0.0]],

         [[    0.0,     0.0,     0.0,     0.0,     0.0],
          [ 1000.0,     0.0,     0.0,     0.0,     0.0],
          [    0.0,  

In [127]:
from itertools import product
for i in product(*[range(x) for x in my_dim]): # simplest way I found to iterate over the indices of tensor entries
    print("neuron #{}".format(i))
    print("approx_neurons[i]: {}".format(apn[i]))
    if not apn[i]:
        if torch.all(A[my_nb_error_terms:][:, i] == 0): 
            print("ok")
        else: 
            nonzero_ind = A[my_nb_error_terms:][:, i].nonzero()
            print(nonzero_ind)
            print(A[my_nb_error_terms:][:, *i])

            # print(A[my_nb_error_terms:][nonzero_ind])
            # print(A[my_nb_error_terms:][:, i][0]) # first channel
            raise UserWarning("error!")
    else:
        print(A[my_nb_error_terms:][:, i])

SyntaxError: invalid syntax (<ipython-input-127-6001507bd595>, line 11)

In [115]:
i = (0, 0)
import numpy as np
npa = np.array([[1, 2, 3], [4, 5, 6]])
npa[i]

1

In [129]:
pta = torch.tensor([
    [[1, 2, 3], [4, 5, 6]], 
    [[11, 22, 33], [44, 55, 66]]
])
j = (0, 0, 0)
print( pta[j] )

print( pta[:, i[0], i[1]] ) # what I want
print( pta[:, i] )          # wtf

tensor(1)
tensor([ 1, 11])
tensor([[[ 1,  2,  3],
         [ 1,  2,  3],
         [ 4,  5,  6]],

        [[11, 22, 33],
         [11, 22, 33],
         [44, 55, 66]]])
